In [5]:
import opendssdirect as dss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random 

In [2]:
print('delta \n ')
fn = "02node_delta.dss"
dss.run_command('Redirect 02node_delta.dss')
for n in range(len(dss.Circuit.AllBusNames())):
    print(dss.Circuit.AllBusNames()[n])
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[n])
    print(dss.Bus.PuVoltage())
    print(dss.Bus.kVBase())
print('\n wye \n')   
fn = "02node_wye.dss"
dss.run_command('Redirect ' + fn)
for n in range(len(dss.Circuit.AllBusNames())):
    print(dss.Circuit.AllBusNames()[n])
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[n])
    print(dss.Bus.PuVoltage())
    print(dss.Bus.kVBase())
    Zbase = dss.Bus.kVBase() * 1000
    

delta 
 
sourcebus
[0.9999994088515389, -3.8125235840150153e-06, -0.49999940764115164, -0.866021594584593, -0.5000000011128107, 0.8660254072771836]
2.7712812921102037
a01
[0.9978748186283304, 0.00484700930260567, -0.4978782291711277, -0.8708693910183178, -0.4999965893596269, 0.8660223818847196]
2.7712812921102037
a02
[0.989386035047192, 0.024253540817513068, -0.4893955108954092, -0.890270544072074, -0.4999905240542071, 0.86601700342357]
2.7712812921102037

 wye 

sourcebus
[0.9999989981247389, -1.2808023152535874e-06, -0.5000004679777752, -0.8660238136984597, -0.4999999167869648, 0.8660254305494991]
2.7712812921102037
a01
[0.9973919359494563, 0.002426623301730239, -0.5011103025237464, -0.8673146714626164, -0.5029006195958526, 0.8661393291274035]
2.7712812921102037
a02
[0.9869733185916489, 0.012141448194922603, -0.5055516278558754, -0.8724880569404879, -0.5145109899129376, 0.8666016274482832]
2.7712812921102037


In [27]:
print( random.randrange(0, 300))

144


In [4]:
dss.run_command('Redirect 02node_wye.dss') #wye has two single phase loads
dss.Loads.Name(dss.Loads.AllNames()[0])
wyep0 = dss.Loads.kW()
wyeq0 = dss.Loads.kvar()

print(wyep0)
print(dss.CktElement.Currents())

100.0
[36.780377730244595, -17.827496865209934, -36.780377730244595, 17.827496865209934]


In [55]:
# https://www.geeksforgeeks.org/genetic-algorithms/

dss.run_command('Redirect 02node_delta.dss')
# Number of individuals in each generation 
POPULATION_SIZE = 4

Vbase = 2.7712812921102037 * 1000
Sbase = 1000000.0
Ibase = Sbase / Vbase
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

class Bus(object): 
    ''' 
    Class representing individual in population 
    '''
    def __init__(self, wyep0, wyeq0, wyep1, wyeq1):#, deltap, deltaq):
       
        self.wyep0 = wyep0
        self.wyeq0 = wyeq0
        self.wyep1 = wyep1
        self.wyeq1 = wyeq1
        self.fitness = self.cal_fitness() 

    @classmethod
    def mutations(self): 
        ''' 
        create random vals for kwatt/kvar for wye/delta
        '''       
        return random.randrange(0, 300),random.randrange(0, 300),random.randrange(0, 300),random.randrange(0,300)


    def mate(self, par2): 
        ''' 
        Perform mating and produce new offspring 
        '''
              
        prob = random.random()

        if prob < 0.25: #parent 1
            child_chromosome_wyep0 = (self.wyep0) 
            child_chromosome_wyeq0 = (self.wyeq0) 
            child_chromosome_wyep1 = (self.wyep1) 
            child_chromosome_wyeq1 = (self.wyeq1) 
        elif prob < 0.40: #parent 2
            child_chromosome_wyep0 = (par2.wyep0) 
            child_chromosome_wyeq0 = (par2.wyeq0) 
            child_chromosome_wyep1 = (par2.wyep1) 
            child_chromosome_wyeq1 = (par2.wyeq1) 
        elif prob < 0.65: # mixed
            child_chromosome_wyep0 = (self.wyep0) 
            child_chromosome_wyeq0 = (self.wyeq0) 
            child_chromosome_wyep1 = (par2.wyep1) 
            child_chromosome_wyeq1 = (par2.wyeq1) 
        elif prob < .8: # half chaos
            child_chromosome_wyep0, child_chromosome_wyeq0, child_chromosome_wyep1, child_chromosome_wyeq1= self.mutations()
            child_chromosome_wyep1 = (par2.wyep1) 
            child_chromosome_wyeq0 = (self.wyeq0) 
        else: # total chaos
            child_chromosome_wyep0, child_chromosome_wyeq0, child_chromosome_wyep1, child_chromosome_wyeq1= self.mutations() 
        
        return Bus(child_chromosome_wyep0, child_chromosome_wyeq0, child_chromosome_wyep1, child_chromosome_wyeq1)
    
    @classmethod
    def create_gnome(self): 
        ''' 
        create initial wye 
        '''
        #retrieve wye loads
        dss.run_command('Redirect 02node_wye.dss') #wye has two single phase loads
        dss.Loads.Name(dss.Loads.AllNames()[0])
        wyep0 = dss.Loads.kW()
        wyeq0 = dss.Loads.kvar()
        
        dss.Loads.Name(dss.Loads.AllNames()[1])
        wyep1 = dss.Loads.kW()
        wyeq1 = dss.Loads.kvar()
             
        return wyep0, wyeq0, wyep1, wyeq1
        

    def cal_fitness(self): 
        ''' 
        Calculate fitness score, the 2 norm of the difference in delta and wye
        current and voltage
        '''
        
        #set the power for the nodes and then dss solve and then 
        #two norm
        
        #retrieve delta voltages and currents
        V_delta = np.array([])
        I_delta = np.array([])
        
        dss.run_command('Redirect 02node_delta.dss')    
        dss.Solution.Solve()
        
        for k1 in range(1, 2):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
            Imn = Imn[0:int(len(Imn)/2)]
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
            I_delta = np.append(I_delta, Imn)

        for k2 in range(1, 3):
            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])
            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]   
            V_delta = np.append(V_delta, Vtemp)

        #retrieve wye voltages and currents
        V_wye = np.array([])
        I_wye = np.array([])
        
        dss.run_command('Redirect 02node_wye.dss')
        dss.Loads.Name(dss.Loads.AllNames()[0])
        dss.Loads.kW(self.wyep0)
        dss.Loads.kvar(self.wyeq0) #reset 
        dss.Loads.Name(dss.Loads.AllNames()[1])
        dss.Loads.kW(self.wyep1)
        dss.Loads.kvar(self.wyeq1) #reset 
        dss.Solution.Solve()
        
        for k1 in range(1, 2):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
            Imn = Imn[0:int(len(Imn)/2)]
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
            I_wye = np.append(I_wye, Imn)
        
        for k2 in range(1, 3):
            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])
            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
            V_wye = np.append(V_wye, Vtemp)
        
        #calculate 2 norm
        fitness = np.linalg.norm(np.abs(V_delta - V_wye)) + np.linalg.norm(np.abs(I_delta - I_wye))
        
        return fitness 
    
    def solve_with_loads(self):
        dss.run_command('Redirect 02node_delta.dss')
       
        dss.Solution.Solve()
   
        print('line currents of delta')
        for k1 in range(1, 2):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
            Imn = Imn[0:int(len(Imn)/2)]
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
            print(Imn)
 
        print('bus voltages of delta')
        for k2 in range(1, 3):
            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])
            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
            print(Vtemp)
            
        print('delta loads')
        for l in range(len(dss.Loads.AllNames())):
            dss.Loads.Name(dss.Loads.AllNames()[l])
            print(dss.CktElement.Powers())
         
        
        dss.run_command('Redirect 02node_wye.dss')
        dss.Loads.Name(dss.Loads.AllNames()[0])
        dss.Loads.kW(self.wyep0)
        dss.Loads.kvar(self.wyeq0) #reset 
        dss.Loads.Name(dss.Loads.AllNames()[1])
        dss.Loads.kW(self.wyep1)
        dss.Loads.kvar(self.wyeq1) #reset 
        dss.Solution.Solve()
             
        print('wye currents')
        for k1 in range(1, 2):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
            Imn = Imn[0:int(len(Imn)/2)]
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
            print(Imn)
        print('wye voltages')
        for k2 in range(1, 3):
            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k2])
            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
            Vtemp = np.asarray(dss.Bus.PuVoltage())
            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
            print(Vtemp)
        print('loads of wye ')
        for l in range(len(dss.Loads.AllNames())):
            dss.Loads.Name(dss.Loads.AllNames()[l])
            print(dss.CktElement.Powers())
       

# Driver code 
def main(): 
    global POPULATION_SIZE 
    global Ibase
    global nnode
    global nline

    #current generation 
    generation = 1

    found = False
    population = [] 

    # create initial population 
    for _ in range(POPULATION_SIZE): 
        wyep0, wyeq0, wyep1, wyeq1 = Bus.create_gnome() 
        population.append(Bus(wyep0, wyeq0, wyep1, wyeq1)) 

    while not found: 

        # sort the population in increasing order of fitness score 
        population = sorted(population, key = lambda x:x.fitness) 

        # if the individual having lowest fitness score ie. 
        # 0 then we know that we have reached to the target 
        # and break the loop 
        print(population[0].fitness)
        if population[0].fitness <= 0.06: 
            print(generation)
            print("\----")
            print('wye 0 real power')
            print(population[0].wyep0)
            print('wye 0 reactive power')
            print(population[0].wyeq0)
            print('wye 1 real power')
            print(population[0].wyep1)
            print('wye 1 reactive power')
            print(population[0].wyeq1)
            
            population[0].cal_fitness()
            population[0].solve_with_loads()
          
            found = True
            
            break

        # Otherwise generate new offsprings for new generation 
        new_generation = [] 

        # Perform Elitism, that mean 10% of fittest population 
        # goes to the next generation 
        s = int((10*POPULATION_SIZE)/100) 
        new_generation.extend(population[:s]) 

        # From 50% of fittest population, Individuals 
        # will mate to produce offspring 
        s = int((90*POPULATION_SIZE)/100) 
        for _ in range(s): 
            parent1 = random.choice(population[:50]) 
            parent2 = random.choice(population[:50]) 
            child = parent1.mate(parent2)       
            new_generation.append(child) 

        population = new_generation 


        generation += 1
        print(generation)


if __name__ == '__main__': 

    main() 
    


0.27050438794876763
2
0.2138240427273203
3
0.10321803782083026
4
0.10321803782083026
5
0.35187714208916737
6
0.3018198264505799
7
0.3018198264505799
8
0.3162385718972535
9
0.29586563477685945
10
0.18743861822352978
11
0.18743861822352978
12
0.3198754632499776
13
0.22131008445729383
14
0.21860673801100233
15
0.19610599372679152
16
0.19610599372679152
17
0.19610599372679152
18
0.19610599372679152
19
0.19610599372679152
20
0.3056087218196221
21
0.3420215661710635
22
0.2638839502825186
23
0.2638839502825186
24
0.2638839502825186
25
0.20215449103498723
26
0.2638839502825186
27
0.11513158385651114
28
0.20183679137113555
29
0.20183679137113555
30
0.20183679137113555
31
0.20183679137113555
32
0.20183679137113555
33
0.2760138635666824
34
0.32869274465186593
35
0.32869274465186593
36
0.32869274465186593
37
0.3046715242587502
38
0.20825226540435515
39
0.20825226540435515
40
0.20825226540435515
41
0.20825226540435515
42
0.10838814742171626
43
0.18847357755166885
44
0.18847357755166885
45
0.2931306

358
0.19643253283317516
359
0.19643253283317516
360
0.15418891846233945
361
0.15418891846233945
362
0.15418891846233945
363
0.08407594107324809
364
0.08407594107324809
365
0.08407594107324809
366
0.14998422980815498
367
0.14998422980815498
368
0.14842308484815794
369
0.14998422980815498
370
0.14998422980815498
371
0.16045133549079504
372
0.1250972278782179
373
0.17792593494872333
374
0.17792593494872333
375
0.2815950643592016
376
0.2273996412315972
377
0.2815950643592016
378
0.2606699914994326
379
0.2446560886861226
380
0.13846038139614875
381
0.2446560886861226
382
0.2446560886861226
383
0.31151827262481646
384
0.31379445787007254
385
0.3082361580192988
386
0.19258259550491386
387
0.3069804751166003
388
0.2707159337787713
389
0.16997165369403533
390
0.16997165369403533
391
0.0802902433489164
392
0.0802902433489164
393
0.0802902433489164
394
0.0802902433489164
395
0.0802902433489164
396
0.0802902433489164
397
0.18858639874285335
398
0.18858639874285335
399
0.26092762715975626
400
0.260

708
0.26596908816047105
709
0.13407217928753215
710
0.13407217928753215
711
0.13407217928753215
712
0.17362497027591997
713
0.15700012824682621
714
0.15700012824682621
715
0.12968736968667208
716
0.1567770689629847
717
0.1567770689629847
718
0.15700012824682621
719
0.15700012824682621
720
0.15700012824682621
721
0.12926315945906658
722
0.12926315945906658
723
0.29291128295196056
724
0.16844210082938546
725
0.16844210082938546
726
0.33725766543494995
727
0.33725766543494995
728
0.33725766543494995
729
0.424345005932652
730
0.2281743487581966
731
0.332879752994218
732
0.30612991131611667
733
0.4076504293986446
734
0.34764398144138386
735
0.34764398144138386
736
0.3592810149024558
737
0.3905902295140235
738
0.3862842403284065
739
0.3678697942390692
740
0.3545194724812881
741
0.3815051207388502
742
0.14848347416693006
743
0.14848347416693006
744
0.11380063878721033
745
0.3198178354809593
746
0.14420213072518548
747
0.14420213072518548
748
0.14420213072518548
749
0.24999901816408024
750
0.2

1055
0.37147348989187057
1056
0.30957596357430184
1057
0.4024455897319548
1058
0.26481416375554334
1059
0.2651154844890231
1060
0.24747475893952547
1061
0.28948950317402505
1062
0.28948950317402505
1063
0.33692196784252
1064
0.23363425262720816
1065
0.22291067543082055
1066
0.17636342658590176
1067
0.2112101678056583
1068
0.2112101678056583
1069
0.2112101678056583
1070
0.2112101678056583
1071
0.2112101678056583
1072
0.2112101678056583
1073
0.2112101678056583
1074
0.2170885481951163
1075
0.2170885481951163
1076
0.2677400308942686
1077
0.2677400308942686
1078
0.2677400308942686
1079
0.2677400308942686
1080
0.2817237197736876
1081
0.25831179665854026
1082
0.25831179665854026
1083
0.25831179665854026
1084
0.2817237197736876
1085
0.2817237197736876
1086
0.2817237197736876
1087
0.21942894019672754
1088
0.19233315054280595
1089
0.2316583716905595
1090
0.33067972134162255
1091
0.3736475393832954
1092
0.3338452317753474
1093
0.39534939504852373
1094
0.3749034450220724
1095
0.2954916071150423
10